## **1. Regroupement en sous-corpus**
Nous allons extraire automatiquement des tags en déconstruisant les URLs des pages aspirées et afin de créer des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

In [ ]:
from pandas import *
import os
acteurs = []
base_path = '../01-corpus/2-data/1-fr/'
for file in os.listdir(base_path):
    if file.endswith('.csv'):
        acteurs.append({'acteur':file[:-4]})

In [ ]:
x = 0 # À modifier manuellement
path = '../01-corpus/2-data/1-fr/'
acteur = acteurs[x]['acteur']

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=';')[["Address", 'Title', 'Type', "text"]]

csv

In [ ]:
liste = [lien.replace('//', "/") for lien in csv['Address'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien and not 'nouvelles' in lien]
print(str(len(liste)) + ' liens.')
liste 

In [ ]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens = []
for i in range (2, 9): # On prend les liens de niveau 2 à 8
    try:
        liens += [lien for lien in liste if len(lien.split('/')) > i]
    except Exception as e:
        True

In [ ]:
regroupements = {}

for lien in liens:
    for i in range(2,9):
        try:
            tag = lien.split('/')[i-1]
            if tag in regroupements:
                regroupements[tag].append(lien)
                
            else:
                if len(tag)>1:
                    regroupements[tag] = []

        except Exception as e:
            True

tags = list(regroupements.keys())
tags = [tag for tag in regroupements if len(regroupements[tag]) > 20]

In [ ]:
tags

In [ ]:
print('Ça nous donne {} tags.'.format(len(tags)))
tags

In [ ]:
import os, shutil
from pathlib import Path
from slugify import slugify

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../01-corpus/2-sous-corpus/'

#Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags:
    file_name =  acteur + "_" + slugify(str(tag).strip('\n'))

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    try:
        data = csv[csv["Address"].str.match('.*[\-\_\/]'+tag+'.*')]
        data.to_csv(folder_path + file_name + ".csv") #encoding='ISO-8859-1'
    except Exception as e:
        print(tag, str(e))
                    

In [ ]:
with open(folder_path + acteur + '_sous-corpus.txt', 'w') as f:
    for file in os.listdir(folder_path):
        f.write(file + '\n')

_____________________________________________________________________________________________________________________

### **Étape d'indexation manuelle**

_____________________________________________________________________________________________________________________

Après avoir manuellement indexé

In [ ]:
from pandas import read_csv
filepath = '../01-corpus/2-sous-corpus/' + acteur + '.csv'
with open(filepath, 'r', encoding = 'utf-8') as f:
    csv = read_csv(f, sep=';')
    
csv = csv[csv['Sous-corpus'] != 'x']
csv = csv[csv['Sous-corpus'] != 'X']

csv

In [ ]:
# tags = list(set([tag for tag in csv['oui/non'].tolist()]))
tags = list(set([tag for tag in csv['Sous-corpus'].tolist()]))

tags

In [ ]:
regroupements = {tag : [] for tag in tags}
for tag in tags:
        tuples = csv[csv['Sous-corpus'] == tag]
        liste = [str(t).strip('\n') for t in tuples["Dossier"].tolist()]
        regroupements[tag] = liste


In [ ]:
regroupements

In [ ]:
import pandas
from slugify import slugify

# Créer de nouveaux CSV contenant les données regroupées selon le tag qui leur est associé
path='../01-corpus/2-sous-corpus/'

for tag in regroupements:

    # Aller chercher les éléments correspondant au sous-corpus dans le csv principal et les copier dans un nouveau CSV 
    dfs = []
    path='../01-corpus/2-sous-corpus/'
    for doc in regroupements[tag]:
        with open(path + doc, 'r', encoding='utf-8') as f:
            csv = read_csv(f)
            dfs.append(csv)

    # Concatenate all data into one DataFrame
    try:
        path='../01-corpus/2-data/1-fr/' + acteur +'/'
        file_path = path + acteur + '_'  + slugify(str(tag)) + '.csv'
        big_frame = pandas.concat(dfs, ignore_index=True)
        big_frame['Sous-corpus'] = tag
        big_frame['Corpus'] = acteur
        big_frame = big_frame[["Corpus", "Sous-corpus",	"Address",	"Title", "Type",	"text"]]
        big_frame.to_csv(file_path)
        print(tag + "\t" + str(len(big_frame)))

    except Exception as e:
        print(tag, str(e))